In [1]:
from PIL import Image
import os
import imageio
import numpy as np
import pandas as pd
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout, LeakyReLU
from tensorflow.keras import optimizers
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2

import tensorflow as tf
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession


In [2]:
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [3]:
slum_source = 'Mumbai data/preproccesed data/slum/tiles/'
baseline_source = 'Mumbai data/preproccesed data/baseline/tiles/'

In [4]:
slums = []
for name in os.listdir(slum_source):
    im = imageio.imread(slum_source + '/' + name)
    im = im[:,:,:3]
    slums.append(im)

baseline = []
for name in os.listdir(baseline_source):
    im = imageio.imread(baseline_source + '/' + name)
    im = im[:,:,:3]
    baseline.append(im)

In [5]:
slums = np.array(slums)
baseline = np.array(baseline)


In [6]:
baseline = baseline[:slums.shape[0]]

In [7]:
slums.shape, baseline.shape

((1600, 80, 160, 3), (1600, 80, 160, 3))

In [8]:
labels = np.concatenate((np.zeros(len(slums)),np.ones(len(baseline))))

In [9]:
data = np.concatenate((slums, baseline))

In [10]:
X = data
y = labels
y = to_categorical(y)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, stratify = y)

In [12]:
adam = optimizers.Adam(lr = .001)

In [13]:
pd.DataFrame(y_test)[0].value_counts(normalize=True)

0.0    0.5
1.0    0.5
Name: 0, dtype: float64

In [14]:
mblnet = MobileNetV2(include_top = True, weights = None, input_shape = (X.shape[1:]), classes = 2)

In [15]:
mblnet.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

In [16]:
mblnet.fit(X_train, y_train, epochs=80, validation_data=(X_train, y_train))

Train on 2400 samples, validate on 2400 samples
Epoch 1/80
2400/2400 [==============================] - 8s 3ms/sample - loss: 0.1621 - accuracy: 0.9408 - val_loss: 0.6968 - val_accuracy: 0.5000
Epoch 2/80
2400/2400 [==============================] - 3s 1ms/sample - loss: 0.0459 - accuracy: 0.9854 - val_loss: 0.6959 - val_accuracy: 0.5000
Epoch 3/80
2400/2400 [==============================] - 3s 1ms/sample - loss: 0.0107 - accuracy: 0.9962 - val_loss: 0.6947 - val_accuracy: 0.5000
Epoch 4/80
2400/2400 [==============================] - 4s 1ms/sample - loss: 0.0079 - accuracy: 0.9962 - val_loss: 0.6934 - val_accuracy: 0.5000
Epoch 5/80
2400/2400 [==============================] - 4s 1ms/sample - loss: 0.0178 - accuracy: 0.9950 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 6/80
2400/2400 [==============================] - 3s 1ms/sample - loss: 0.0237 - accuracy: 0.9912 - val_loss: 0.6942 - val_accuracy: 0.5000
Epoch 7/80
2400/2400 [==============================] - 3s 1ms/sample - loss

2400/2400 [==============================] - 3s 1ms/sample - loss: 0.0039 - accuracy: 0.9983 - val_loss: 1.2179 - val_accuracy: 0.5000
Epoch 57/80
2400/2400 [==============================] - 3s 1ms/sample - loss: 0.0021 - accuracy: 0.9992 - val_loss: 1.2597 - val_accuracy: 0.5000
Epoch 58/80
2400/2400 [==============================] - 3s 1ms/sample - loss: 0.0017 - accuracy: 0.9996 - val_loss: 1.4294 - val_accuracy: 0.5000
Epoch 59/80
2400/2400 [==============================] - 4s 2ms/sample - loss: 2.0850e-04 - accuracy: 1.0000 - val_loss: 1.4900 - val_accuracy: 0.5000
Epoch 60/80
2400/2400 [==============================] - 3s 1ms/sample - loss: 6.1748e-04 - accuracy: 1.0000 - val_loss: 1.5406 - val_accuracy: 0.5000
Epoch 61/80
2400/2400 [==============================] - 4s 2ms/sample - loss: 6.9571e-05 - accuracy: 1.0000 - val_loss: 1.5206 - val_accuracy: 0.5000
Epoch 62/80
2400/2400 [==============================] - 4s 2ms/sample - loss: 9.5663e-05 - accuracy: 1.0000 - val_los

In [17]:
predictions = mblnet.predict(X_train)

In [21]:
mblnet.evaluate(X_train, y_train), mblnet.evaluate(X_test, y_test)

800/800 [==============================] - 0s 262us/sample - loss: 0.1467 - accuracy: 0.9862


([0.0727022377947848, 0.99125], [0.1467220068022698, 0.98625])

In [19]:
for i in predictions:
    print(i)

[9.9999821e-01 1.7561645e-06]
[9.9999964e-01 3.3085442e-07]
[1.0986776e-10 1.0000000e+00]
[9.994374e-01 5.625506e-04]
[1.4291346e-11 1.0000000e+00]
[3.3083145e-13 1.0000000e+00]
[9.9999881e-01 1.1590179e-06]
[9.9999845e-01 1.5725116e-06]
[9.9999976e-01 2.5535044e-07]
[2.0177575e-14 1.0000000e+00]
[9.9999893e-01 1.1233737e-06]
[3.3527847e-11 1.0000000e+00]
[6.9872496e-17 1.0000000e+00]
[9.9999917e-01 8.5682944e-07]
[9.9999940e-01 6.5258746e-07]
[1.9300732e-16 1.0000000e+00]
[9.9999952e-01 4.6856644e-07]
[6.4360565e-14 1.0000000e+00]
[9.9999976e-01 1.8916136e-07]
[9.9999762e-01 2.3447276e-06]
[9.9999964e-01 3.5753874e-07]
[9.9999893e-01 1.0565728e-06]
[0.97563744 0.02436256]
[9.9999917e-01 8.8992311e-07]
[6.667919e-16 1.000000e+00]
[3.178562e-12 1.000000e+00]
[2.2625667e-11 1.0000000e+00]
[0.9955076  0.00449238]
[2.9662088e-14 1.0000000e+00]
[3.4705789e-12 1.0000000e+00]
[1.1038203e-16 1.0000000e+00]
[9.9997985e-01 2.0152569e-05]
[1.9400646e-12 1.0000000e+00]
[2.978427e-17 1.000000e+00]


[9.999964e-01 3.556829e-06]
[9.9999917e-01 8.4888649e-07]
[1.8519976e-14 1.0000000e+00]
[1.3403467e-12 1.0000000e+00]
[1.888236e-15 1.000000e+00]
[1.8588206e-14 1.0000000e+00]
[1.3465484e-11 1.0000000e+00]
[2.878203e-13 1.000000e+00]
[7.841773e-12 1.000000e+00]
[9.9998486e-01 1.5113919e-05]
[2.8071907e-12 1.0000000e+00]
[1.628199e-11 1.000000e+00]
[9.9999702e-01 2.9895416e-06]
[8.7263616e-13 1.0000000e+00]
[1.4663072e-16 1.0000000e+00]
[3.046235e-16 1.000000e+00]
[9.9999869e-01 1.3680161e-06]
[3.8906814e-17 1.0000000e+00]
[9.9998939e-01 1.0587278e-05]
[1.345323e-16 1.000000e+00]
[9.9999952e-01 4.7630584e-07]
[3.62275e-11 1.00000e+00]
[9.9999893e-01 1.0509611e-06]
[9.9998713e-01 1.2929734e-05]
[2.2977977e-15 1.0000000e+00]
[2.140665e-14 1.000000e+00]
[9.9999964e-01 3.4781280e-07]
[9.9999881e-01 1.2120391e-06]
[9.9999833e-01 1.6219240e-06]
[7.498152e-14 1.000000e+00]
[7.475868e-12 1.000000e+00]
[2.946598e-13 1.000000e+00]
[9.9999952e-01 4.7257393e-07]
[9.4011525e-17 1.0000000e+00]
[9.999

[4.634725e-15 1.000000e+00]
[1.6386742e-14 1.0000000e+00]
[9.358537e-17 1.000000e+00]
[5.627195e-16 1.000000e+00]
[9.9999964e-01 4.0285548e-07]
[9.9999630e-01 3.7233922e-06]
[4.1838486e-13 1.0000000e+00]
[1.5793248e-14 1.0000000e+00]
[8.138418e-15 1.000000e+00]
[2.979032e-15 1.000000e+00]
[4.2850956e-15 1.0000000e+00]
[1.1197185e-11 1.0000000e+00]
[9.999976e-01 2.401458e-06]
[1.6995022e-13 1.0000000e+00]
[2.9252588e-12 1.0000000e+00]
[9.9995351e-01 4.6540834e-05]
[7.971544e-15 1.000000e+00]
[9.9999297e-01 7.0518595e-06]
[9.9999869e-01 1.3035301e-06]
[1.524839e-14 1.000000e+00]
[9.9999976e-01 2.9087360e-07]
[9.9999976e-01 2.5200126e-07]
[6.451159e-11 1.000000e+00]
[9.9999905e-01 9.7419081e-07]
[4.3538515e-09 1.0000000e+00]
[0.0081168  0.99188316]
[9.9999928e-01 7.5271686e-07]
[1.5336521e-12 1.0000000e+00]
[2.6034854e-09 1.0000000e+00]
[9.9999917e-01 8.6675260e-07]
[9.9999893e-01 1.0794071e-06]
[3.7395812e-11 1.0000000e+00]
[9.999994e-01 5.564713e-07]
[9.9999797e-01 1.9822892e-06]
[9.999

[0.00174022 0.99825984]
[9.9998987e-01 1.0132048e-05]
[9.9999893e-01 1.1183561e-06]
[6.4498976e-13 1.0000000e+00]
[1.300767e-16 1.000000e+00]
[9.9987686e-01 1.2316783e-04]
[9.999962e-01 3.866763e-06]
[3.9564088e-13 1.0000000e+00]
[4.6687446e-14 1.0000000e+00]
[1.0360959e-11 1.0000000e+00]
[9.999994e-01 6.343138e-07]
[9.999995e-01 4.648819e-07]
[9.9999917e-01 8.0637972e-07]
[9.999994e-01 5.394493e-07]
[9.9999905e-01 9.8152441e-07]
[9.9999857e-01 1.4233256e-06]
[9.9999893e-01 1.0500174e-06]
[9.9999857e-01 1.4410211e-06]
[7.64659e-15 1.00000e+00]
[9.9999809e-01 1.9547012e-06]
[8.41019e-15 1.00000e+00]
[1.6650574e-14 1.0000000e+00]
[0.925114   0.07488605]
[9.9999940e-01 5.8655377e-07]
[2.7926767e-11 1.0000000e+00]
[2.836327e-17 1.000000e+00]
[2.6900692e-11 1.0000000e+00]
[0.9190074  0.08099261]
[5.9549824e-17 1.0000000e+00]
[9.026633e-16 1.000000e+00]
[2.9803752e-14 1.0000000e+00]
[0.43052366 0.56947637]
[1.6072872e-16 1.0000000e+00]
[9.9999881e-01 1.2285949e-06]
[3.9513765e-14 1.0000000e+

[9.9999785e-01 2.1700503e-06]
[9.999994e-01 6.306970e-07]
[3.0352953e-13 1.0000000e+00]
[9.699454e-12 1.000000e+00]
[9.9999964e-01 3.6940042e-07]
[9.9999928e-01 6.9234426e-07]
[9.999993e-01 7.588794e-07]
[2.1442038e-14 1.0000000e+00]
[1.3418207e-11 1.0000000e+00]
[9.9999964e-01 3.0850143e-07]
[1.4606033e-14 1.0000000e+00]
[9.9999988e-01 1.4313416e-07]
[5.3183827e-15 1.0000000e+00]
[1.0323542e-14 1.0000000e+00]
[9.999994e-01 5.432335e-07]
[1.0675891e-15 1.0000000e+00]
[2.5782148e-16 1.0000000e+00]
[9.9996221e-01 3.7741574e-05]
[9.999993e-01 7.274690e-07]
[1.7064762e-13 1.0000000e+00]
[1.3611657e-16 1.0000000e+00]
[4.7104456e-07 9.9999952e-01]
[9.9999905e-01 9.5512598e-07]
[7.64876e-12 1.00000e+00]
[6.136695e-13 1.000000e+00]
[9.9999952e-01 4.7619545e-07]
[9.9999905e-01 9.1769692e-07]
[9.9999952e-01 4.3270973e-07]
[4.0570872e-18 1.0000000e+00]
[9.999993e-01 7.518997e-07]
[9.9999857e-01 1.4035447e-06]
[5.202883e-13 1.000000e+00]
[9.999994e-01 6.006078e-07]
[1.2413452e-12 1.0000000e+00]
[4

[9.9999654e-01 3.4016398e-06]
[2.1984773e-16 1.0000000e+00]
[7.137759e-11 1.000000e+00]
[1.7699334e-15 1.0000000e+00]
[9.9999928e-01 6.6198857e-07]
[2.8768887e-11 1.0000000e+00]
[9.9999905e-01 9.1822045e-07]
[1.554125e-14 1.000000e+00]
[9.9999964e-01 3.6678904e-07]
[2.6529794e-11 1.0000000e+00]
[1.3948649e-16 1.0000000e+00]
[9.999980e-01 1.977397e-06]
[9.59276e-16 1.00000e+00]
[9.999994e-01 5.836778e-07]
[1.0624008e-12 1.0000000e+00]
[6.870072e-13 1.000000e+00]
[9.9999893e-01 1.0485654e-06]
[9.9999928e-01 7.4897395e-07]
[9.9906868e-01 9.3130715e-04]
[3.690545e-16 1.000000e+00]
[9.9999738e-01 2.6418527e-06]
[1.6682363e-14 1.0000000e+00]
[9.994910e-01 5.090141e-04]
[9.9999940e-01 6.2355906e-07]
[9.9999678e-01 3.2693922e-06]
[9.9999905e-01 1.0053490e-06]
[1.4705764e-15 1.0000000e+00]
[9.9999774e-01 2.2444524e-06]
[2.5258837e-13 1.0000000e+00]
[7.883508e-16 1.000000e+00]
[9.9999774e-01 2.2613806e-06]
[9.999118e-01 8.815576e-05]
[9.9999952e-01 5.1031293e-07]
[8.402165e-17 1.000000e+00]
[9.9

In [22]:
mblnet.save('stored_image_classifier')

INFO:tensorflow:Assets written to: stored_image_classifier/assets
